<strong><h5>START WITH IMPORTING LIBRARIES</h5></strong>

In [ ]:
%pip install mediapipe opencv-python pandas scikit-learn

import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import csv # Import csv
import os # Import sperating system
import numpy as np # Import numpy

<strong><h5>MAKE THE HEADERS OR COLUMN NAMES</h5></strong>

In [ ]:
# Number of coordinates to detect
num_coords = 33

# Create the column names
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

# Export to csv
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

<strong><h5>CAPTURE AND EXPORT COORDS TO CSV</h5></strong>

In [ ]:
# Initializes MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# Sets the class name
class_name = "pick"

# Path to the video dataset
video_dir = 'C:/Users/evand/Mega/Projects/Thesis/python/pick/clean'  
video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.mkv', '.avi'))]
total_videos = len(video_files)

# Configures the display window
cv2.namedWindow('Processing Video Feed', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Processing Video Feed', cv2.WND_PROP_TOPMOST, 1)  
cv2.resizeWindow('Processing Video Feed', 1280, 720) 

# Loop through the entire video dataset
try:
    for i, video_file in enumerate(video_files, start=1):
        cap = cv2.VideoCapture(os.path.join(video_dir, video_file))

        # handle file error
        if not cap.isOpened():
            print(f"Error: Could not open video file {video_file}")
            continue

        # print status in CLI
        print(f"Processing {i}/{total_videos}: {video_file}")

        # Perform holistic human pose estimation
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                ret, frame = cap.read()

                # display each video succession
                if not ret:
                    print(f"End of video file: {video_file}")
                    break

                # Maintain aspect ratio
                height, width = frame.shape[:2]
                aspect_ratio = width / height
                new_width = 1280  # Define a new width
                new_height = int(new_width / aspect_ratio)
                frame = cv2.resize(frame, (new_width, new_height))

                # Recolor feed
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False        

                # Perform detection
                results = holistic.process(image)

                # Recolor image back to BGR for rendering
                image.flags.writeable = True   
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Draw landmarks on the frame
                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))

                mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

                # Draw the progress bar
                progress = int((i / total_videos) * new_width)
                progress_bar_height = 5  
                cv2.rectangle(image, (10, new_height - progress_bar_height - 5), (10 + progress, new_height - 5), (0, 255, 0), -1)

                # Export coordinates to CSV
                try:
                    if results.pose_landmarks:
                        pose = results.pose_landmarks.landmark
                        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

                        # Concatenate rows
                        row = pose_row

                        # Append class name 
                        row.insert(0, class_name)  

                        # Export to CSV
                        with open('coords.csv', mode='a', newline='') as f:
                            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                            csv_writer.writerow(row)
                            f.flush()  # Ensure data is written to the file

                except Exception as e:
                    print(f"Error writing to CSV for {video_file}: {e}")
                    break

                # Display the image in the window
                cv2.imshow('Processing Video Feed', image)

                # Small delay to allow for window events
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

                # Check if the window was closed
                if cv2.getWindowProperty('Processing Video Feed', cv2.WND_PROP_VISIBLE) < 1:
                    break

        cap.release()

except KeyboardInterrupt:
    print("Process interrupted by user.")

finally:
    # Ensure the window is closed properly
    cv2.destroyAllWindows()
    print("Processing Finished.")

<strong><h5>COORDINATES CHART</h5></strong>

<img src="https://i.imgur.com/AzKNp7A.png">